In [6]:
from utils import generate_prompt

base = '''The next line is a clue for a cryptic crossword. The clue consists of a definition  part, and a wordplay part. The answer consists of {n_words} words, and number of characters in the answer are {n_chars}. Output only the answer.
'''

In [7]:
base.format(n_words='2', n_chars='10')

'The next line is a clue for a cryptic crossword. The clue consists of a definition  part, and a wordplay part. The answer consists of 2 words, and number of characters in the answer are 10. Output only the answer.\n'

In [8]:
import re
def get_ans_words_chard(clue):
    # Regular expression to match strings inside parentheses
    pattern = r'\((.*?)\)'
    # Find all matches
    matches = re.findall(pattern, clue)[-1]

    numbers = matches.split(',')

    return len(numbers), matches


In [4]:
get_ans_words_chard('what has little possibility of happening - a hole in one? (4,4)')

(2, '4,4')

In [9]:
generate_prompt({'clue': 'what has little possibility of happening - a hole in one? (4,5,6,1,2)', 'labels': 'asd'}, base, True)

{'clue': 'what has little possibility of happening - a hole in one? (4,5,6,1,2)',
 'labels': 'asd',
 'prompt': '### Instruction: The next line is a clue for a cryptic crossword. The clue consists of a definition  part, and a wordplay part. The answer consists of 5 words, and number of characters in the answer are 4,5,6,1,2. Output only the answer.\n\n\n### Input:\nwhat has little possibility of happening - a hole in one? (4,5,6,1,2)\n\n### Response:\nasd'}